# 第1题

为了保证不上溢、下溢。对原公式做出如下修改
\begin{align*}
log\sum_{i=1}^Ne^{x_i}&=log(e^{x_1}+e^{x_2}+...+e^{x_N})\\
&=log(e^a·(e^{x_1-a}+e^{x_2-a}+...+e^{x_N-a}))\\
&=a+\log \sum_{i=1}^N e^{x_i-a}\\
\\
\end{align*}
其中，a的值取
$$a=\max_{i\in [N]} x_i$$
保证了e的指数最大为0，不会上溢.下溢的结果会被认为是0（上溢会报错，或变成负数）

In [11]:
import numpy as np

test_data=[[99,1000,100000],[-10,-1000,-20000],[1,2,3],[20,23,5]]
def compute(data):
    result = []
    for this_data in data:
        a = np.array(max(this_data))
        this_nd=np.array(this_data)
        result.append(a + np.log(sum(np.exp(this_data-a)))) # numpy自动广播
    return result

# bug：np.e和np.exp不同，np.e只能对单个元素操作。np.exp可以对array来操作
if __name__ == "__main__":
    print(compute(test_data))

[100000.0, -10.0, 3.4076059644443806, 23.048587366081428]


# 第二题



In [2]:
# 以下将按照word文档中网络结构构建

import torch
from torch import nn

net = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=9, padding=0), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2,padding=0),
    nn.Conv2d(32, 64, kernel_size=5, padding=0), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2,padding=0),
    nn.Conv2d(64, 64, kernel_size=5, padding=0), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2,padding=0),
    nn.Flatten(),
    nn.Linear(12*12*64, 3), nn.ReLU(),# 一层FC网络，3个神经元，以ReLU()作为激励函数
)

# 以下检验网络建构正确性，给出了一个随机化样本，满足输入维度要求
X = torch.rand(size=(1, 3, 128, 128), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Conv2d output shape: 	 torch.Size([1, 32, 120, 120])
ReLU output shape: 	 torch.Size([1, 32, 120, 120])
MaxPool2d output shape: 	 torch.Size([1, 32, 60, 60])
Conv2d output shape: 	 torch.Size([1, 64, 56, 56])
ReLU output shape: 	 torch.Size([1, 64, 56, 56])
MaxPool2d output shape: 	 torch.Size([1, 64, 28, 28])
Conv2d output shape: 	 torch.Size([1, 64, 24, 24])
ReLU output shape: 	 torch.Size([1, 64, 24, 24])
MaxPool2d output shape: 	 torch.Size([1, 64, 12, 12])
Flatten output shape: 	 torch.Size([1, 9216])
Linear output shape: 	 torch.Size([1, 3])
ReLU output shape: 	 torch.Size([1, 3])


# 第三题

1. **ELBO loss function**


**解**：
ELBO 也即 evidence lower bound,也称 variational lower bound。

可以认为是在$Q_\Phi$不断优化过程中（Q和P越来越接近），VAE导出的likelihood的下界


此时模型进行变分推断，使用高斯混合分布 和 EM算法
- 预估高斯混合分布各参数取值
- 在此高斯混合分布上进行采样，产生新的样本

step1：原样本x在通过encoder后，依据两输出$\mu^\prime(x),\sigma^\prime(x)$ sample出z

step2:此后需要把z过decoder，生成$\mu(z),\sigma(z)$来重新产生x，使得在高斯混合分布上sample出的x的似然最大，进而确定x所处的具体正态分布参数$\mu(z),\sigma(z)$。

也即
$$\max L=\max \sum_x \log P_\theta(x)$$

以下先给出ELBO loss function的代数推导
$$\begin{align*}
\log P_\theta(x)&=\log P_\theta(x)·\int_z P_\theta(z|x)dz \\
&=\log P_\theta(x)·\int_z Q_\Phi (z|x)dz \\
&=\int_z Q_\Phi (z|x)·\log P_\theta(x)dz \\
&=\int_z Q_\Phi (z|x)·\log\left(  \frac{P_\theta(z,x)}{P_\theta(z|x)}  \right) dz  \\
&=\int_z Q_\Phi (z|x)· \log\left(  \frac{P_\theta(z,x)}{Q_\Phi (z|x)}·\frac{Q_\Phi (z|x)}{P_\theta (z|x)}  \right) dz \\
&=\int_z Q_\Phi (z|x)·\log\left(  \frac{P_\theta(z,x)}{Q_\Phi (z|x) }\right)+\int_z Q_\Phi(z|x) \log\left( \frac{Q_\Phi (z|x)}{P_\theta (z|x)}  \right)\\
&=\int_z Q_\Phi (z|x)·\log\left(  \frac{P_\theta(z,x)}{Q_\Phi (z|x) }\right)+KL\left(Q_\Phi (z|x)|P_\theta(z|x)\right)\\
&=\mathcal L(\theta,\Phi,x)+KL\left(Q_\Phi (z|x)|P_\theta(z|x)\right)
\end{align*}$$
其中$\mathcal L(\theta,\Phi,x)$即为需要简化的ELBO loss,化简过程如下

$$\begin{align*}
\mathcal L(\theta,\Phi,x)&=\int_zQ_\Phi(z|x)·\log\left(  \frac{P_\theta(z,x)}{Q_\Phi(z|x)}  \right)\\
&=\int_zQ_\Phi(z|x)·\log\left(  \frac{P_\theta(x|z)·P_\theta(z)}{Q_\Phi(z|x)}  \right)\\
&=\int_zQ_\Phi(z|x)·\log\left(  \frac{P_\theta(z)}{Q_\Phi(z|x)}  \right)+\int_zQ_\Phi(z|x)·\log\left(  P_\theta(x|z)  \right)\\
&=-KL(Q_\Phi(z|x)||P_\theta(z))+\mathbb{E}_{z-Q_\Phi(z|x)}[\log P_\theta (x|z)]  \\
\\
\\
\end{align*}$$
此式即为化简后的ELBO loss

---
---


1. **为什么优化ELBO loss function能够“Maximize Likelihood”？**

在推导过程中，可以发现引入的$Q_\Phi$可以是任意函数
- $Q_\Phi$的引入不会对$\log P_\theta(x)$的值造成影响
- $Q_\Phi$的引入会同时影响$\mathcal L(\theta,\Phi,x)$和$KL\left(Q_\Phi (z|x)|P_\theta(z|x)\right)$

而由于$$\log P_\theta(x)=\mathcal L(\theta,\Phi,x)+ KL\left(Q_\Phi (z|x)|P_\theta(z|x)\right)$$

所以在只有$Q_\Phi$变化的条件下，$$\max_{Q_\Phi} \mathcal L(\theta,\Phi,x)\equiv \min_{Q_\Phi}KL\left(Q_\Phi (z|x)|P_\theta(z|x)\right)$$

所以由于KL散度的定义，当$Q_\Phi (z|x)$和$P_\theta(z|x)$尽可能接近相似时，$\mathcal L(\theta,\Phi,x)$和$\log P_\theta(x)$的函数间隙越来越小，也就可以认为
$$\max_{Q_\Phi} \mathcal L(\theta,\Phi,x)\equiv \max \log P_\theta(x)$$
也即优化ELBO loss function能够“Maximize Likelihood”

